## prerequisites:

- Beautifulsoup
- selenium
- pandas
- webdriver manager
- openpyxl

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import urllib3
import re
from bs4 import BeautifulSoup
import csv
http=urllib3.PoolManager()
import requests
import json

# Scraping all the links of hackathons

Set ScrollNumber variable in code to any number to gather more links of hackathons hosted on this site

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())

#The website that consists of all the hackathons and their links
driver.get("https://unstop.com/hackathons?filters=,all,all,all&types=teamsize,payment,eligible,oppstatus")

#for just testing purposes the number of scrolls are set to 3, we can increase it any number depending on the 
# hackathons we want to extract contact information for.
ScrollNumber = 100
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    time.sleep(5)

# DS.html has the html code for the infinite scrolling page    
file = open('DS.html', 'w')
file.write(driver.page_source)
file.close()

driver.close()

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
INFO:WDM:Current google-chrome version is 96.0.4664
[WDM] - Get LATEST chromedriver version for 96.0.4664 google-chrome
INFO:WDM:Get LATEST chromedriver version for 96.0.4664 google-chrome
[WDM] - Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache
INFO:WDM:Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


## Scrape the HTML code and extract links for all the hackathons and output to excel

In [26]:
data = open('DS.html','r')
soup = BeautifulSoup(data, 'html.parser')
for college_name in soup.find_all('h3', {'class': 'double-wrap ng-star-inserted'}):
    print(college_name.text)

 Graphic Era Hill University (GEHU), Bhimtal 
 Queen City Hacks 
 Rock Hacks 
 Jaypee Institute of Information Technology (JIIT), Noida 
 Indian Institute of Information Technology (IIIT), Bhagalpur 
 Indian Institute of Information Technology (IIIT), Bhagalpur 
 Indian Institute of Information Technology (IIIT), Bhagalpur 
 Netaji Subhas University of Technology (NSUT), Delhi 
 The NorthCap University (NCU), Gurgaon 
 Indian Institute of Technology (IIT), Bhubaneswar 
 Indian Institute of Technology (IIT), Bhubaneswar 
 Indian Institute of Information Technology (IIIT), Bhagalpur 
 Mount Holyoke 
 Showcode 
 Unihacks 
 Hack Club 
 Niural 
 Taronga 
 Birla Institute of Technology & Science (BITS), Pilani 
 Bharati Vidyapeeth's College of Engineering (BVCOE), New Delhi 
 International Institute of Information Technology (IIIT), Naya Raipur 
 Praxis Business School, Kolkata 
 Birsa Institute of Technology (BIT), Sindri 
 Pnapaao Private Limited 
 Indian Institute of Technology (IIT), Del

In [113]:
def scrape(file_name='DS.html'):
    contact_link_list = []
    contacts_json_list = []
    college_name_list = []
    data = open(file_name,'r')
    soup = BeautifulSoup(data, 'html.parser')
    # find according to regex (listing order-\d id-[0-9]+ full_banner ng-star-inserted)
    for link in soup.find_all('a', {'class': re.compile(r"listing order-[A-Za-z0-9]+ id-[0-9]+", re.IGNORECASE)}):
        print('https://unstop.com'+link.get('href'))
        contact_link, contacts_json = extract_contacts('https://unstop.com'+link.get('href'))
        contact_link_list.append(contact_link)
        contacts_json_list.append(contacts_json)
    for college_name in soup.find_all('h3', {'class': 'double-wrap ng-star-inserted'}):
        college_name_list.append(college_name.text)
                
    return contact_link_list, contacts_json_list, college_name_list

# Using the API to extract contacts:
def extract_contacts(link):
    
    contact_link = ''
    contacts_json = ''
    non_contact_link = ''
    
    
    last_bit= link.split('/')[-1].split('-')[-1]
    response_API = requests.get('https://unstop.com/api/public/competition/'+last_bit)
    if last_bit.isdigit():
        data = response_API.text
        parse_json = json.loads(data)
        if 'data' in parse_json and 'competition' in parse_json['data']:
            if 'contacts' in parse_json['data']['competition']:
                contact_link = link
                contacts_json = parse_json['data']['competition']['contacts']
                return contact_link, contacts_json
            else:
                non_contact_link = link
                contacts_json = ''
                return non_contact_link, contacts_json
    else:
        return '',''

contact_link_list, contacts_json_list, college_name_list = scrape('DS.html')
length = [len(i) for i in contacts_json_list]
df = pd.DataFrame({'hackathon links': contact_link_list, 'Contact_information': contacts_json_list, 'college_name': college_name_list, 'length':length})
# clearing up the jsonand forming sepeate columns for the contact infromations
lol = []
for i in df['Contact_information']:
    for j in i:
        temp.append([j['name'],j['contact_no'],j['email']])
    lol.append(temp)
    temp = []

contact_info_df = pd.DataFrame(lol) 
cols_number = len(contact_info_df.columns)
contact_info_df_new = pd.DataFrame()
for i in contact_info_df.columns:
    contact_info_df_new = pd.concat([contact_info_df_new, contact_info_df[i].apply(pd.Series)], axis=1)
cols_name = []
for i in range(int(len(contact_info_df_new.columns)/3)):
    cols_name.extend(['Name_'+str(i), 'Number_'+str(i), 'Email_'+str(i)])
contact_info_df_new.columns = cols_name
    
for i in contact_info_df_new.columns:
    df[i] = contact_info_df_new[i]
columns_to_keep = ['hackathon links','college_name'] + [i for i in contact_info_df_new.columns]
contacts_df = df[df['length']!=0][columns_to_keep]
contacts_df = contacts_df.sort_values(by='college_name')
non_contacts_df = df[df['length']==0][['hackathon links', 'college_name']]

contacts_df.to_excel('contacts.xlsx', sheet_name='sheet1', index=False)
non_contacts_df.to_excel('non_contacts.xlsx', sheet_name='sheet1', index=False)


In [ ]:
## Using the api method instead of scrapping method(so the code below is not useful for now)

# ## Trying to figure out how to extract contact information:

# ### Stuck here: [created a forum link](https://forum.webscraper.io/t/extract-contact-information-from-website-but-stuck/10543)

# import os
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# import time
# from webdriver_manager.chrome import ChromeDriverManager
# import pandas as pd

# driver = webdriver.Chrome(ChromeDriverManager().install())

# driver.get("https://unstop.com/hackathon/weave-the-web-powered-by-edot-solutions-techyon-2022-padre-conceicao-college-of-engineering-salcete-goa-462438")
# stages = driver.find_elements_by_xpath("/html/body/div[1]/main/app-public-competition/div/div[2]/div[2]/div/div[1]/div[3]/div/app-dates-and-contacts/section/div/div[2]/ul")
# print(stages)


# ## For contacts extract all of the content under the class "contact ng-star-inserted"

# select only the tech ones

In [75]:
df1 = pd.read_excel('827 tech, to be cleaned.xlsx',engine='openpyxl')

In [78]:
df1.shape, df1.columns

((1847, 6),
 Index(['PHONE', 'COLLEGE NAME', 'STATE', 'DISTRICT', 'MAIL ID', 'PHONE.1'], dtype='object'))

In [85]:
df2 = df1[df1['COLLEGE NAME'].str.contains('tech', case=False)]

In [86]:
df2.to_excel('only_tech.xlsx', sheet_name='sheet1', index=False)